<a href="https://colab.research.google.com/github/daichi-a/PythonTextAnalysis/blob/main/%E6%97%A5%E6%9C%AC%E8%AA%9EBERT%E3%81%AE%E7%89%B9%E5%BE%B4%E9%87%8F%E7%AE%97%E5%87%BA%E3%81%A82%E9%A0%85%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://qiita.com/karaage0703/items/30485c2ba1c396760982
このページの次の情報を基に


https://qiita.com/m__k/items/50b018c60f952c28869e
ここではMeCabのバージョンによって落ちる問題が報告されている。

In [ ]:
# 必要なライブラリのpip

!apt install aptitude swig
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.996.5
!pip install unidic-lite
!pip install transformers

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
  swig3.0
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-locale-perl libfcgi-perl
  libhtml-parser-perl libhtml-tagset-perl libhttp

In [ ]:
# 東北大のモデルに必要なfugashiとIPA辞書
!pip install fugashi
!pip install ipadic

     |████████████████████████████████| 13.4MB 282kB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-cp37-none-any.whl size=13556725 sha256=23da3c8bbc8151df0a91ae8a04d15830fa09dd89ecede1acfb756e116eecbbcd
  Stored in directory: /root/.cache/pip/wheels/ff/00/d1/0c094a0ce58a77199a0c5801f0ecf510c80f0ecbec27f07d2c
Successfully built ipadic


In [ ]:
# 特徴量抽出に必要なライブラリのimportとモデルのロードと構築
import torch
from transformers import BertJapaneseTokenizer, BertModel

# 東北大の。Tokenizerの形態素解析にMeCabを使用（MeCabを経由しているだけだが）
bert_tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
model_bert = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# 訓練モードに入る
# model_bert.train()

# predictモードに入る
model_bert.eval()


# 京大の学習済みモデル
# https://nlp.ist.i.kyoto-u.ac.jp/index.php?ku_bert_japanese
# ここからダウンロードして使用する必要がある
# こちらは形態素解析にJuman++を使用する前提
# このJuman++は、Pythonラッパーは存在するが本体がaptで入らないという問題がある
#bert_tokenizer = BertTokenizer("bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers/vocab.txt",
#                               do_lower_case=False, do_basic_tokenize=False)
#model_bert = BertModel.from_pretrained('bert/Japanese_L-12_H-768_A-12_E-30_BPE_transformers')


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
# 特徴量抽出の関数の定義
# やってることは最終層の出力を見るだけ
# 最終層のレイヤーの出力。これをそのまま使うのは良くないと公式ドキュメントにあるそうな
# 「隠れ層を平均するなりプーリングするなりして使ってくれ」
import numpy as np

def calc_embedding_last_layer(text):
  bert_tokens = bert_tokenizer.tokenize(text)
  ids = bert_tokenizer.convert_tokens_to_ids(["[CLS]"] + bert_tokens[:126] + ["[SEP]"])
  tokens_tensor = torch.tensor(ids).reshape(1, -1)

  with torch.no_grad():
    output = model_bert(tokens_tensor)
    return output[1].numpy() # これが最終層のみの出力を指定している部分か


In [ ]:
# こちらがプーリングも含めた関数定義
# https://qiita.com/yuki_uchida/items/09fda4c5c608a9f53d2f

def compute_vector(text):
    bert_tokens = bert_tokenizer.tokenize(text)
    ids = bert_tokenizer.convert_tokens_to_ids(["[CLS]"] + bert_tokens[:126] + ["[SEP]"])
    tokens_tensor = torch.tensor(ids).reshape(1, -1)

    with torch.no_grad():
      all_encoder_layers = model_bert(tokens_tensor)
      pooling_layer = -2 #この-2は、出力層のひとつ前の隠れ層の出力ということらしい
      # -1は出力層
      # https://github.com/yagays/pytorch_bert_japanese

      #print(_)
      #embedding = all_encoder_layers[0][r].numpy()
      #embedding = all_encoder_layers[0].numpy()
      #return np.mean(embedding, axis=0)
      #return embedding

      output = model_bert(tokens_tensor)
      print(output.pooler_output.shape)
      return output.pooler_output[0].numpy


In [ ]:
# 特徴量の抽出
text = '私はからあげです'
last_layer_feature_vec = calc_embedding_last_layer(text)
two_layer_poling_fecture_vec = compute_vector(text)


torch.Size([1, 768])


In [ ]:
# http://projector.tensorflow.org/
# Enbeddeing Projector
# 勝手に3次元までまで主成分分析してくれる

In [ ]:
# https://propen.dream-target.jp/blog/bert-2
# BertForSequenceClassification

import torch
import random
random.seed(42)

#以下は、torchでGPU使うための設定
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
#ここでは、訓練データ(文字列)とラベルを設定する
#ラベルの意味は以下の通りとする
#0・・・ネガではない(負例)
#1・・・ネガ(正例)

#教師データとしてデータとラベルを設定する
#本来は訓練用データとしてCSVファイルを用意するが、簡単な実装例を説明したいので、直接配列にデータをセットするようにした
train_text_batch = ["この電話は、かなり不便だ。", "この電話は使いやすい。","このラーメン屋のラーメンはまずい","リゲルに行きたい","あいつに負けたぜ"]
train_labels = [1, 0, 1, 0, 1]


#予め用意した教師データ(csv)を読み込む
#csvファイルを予めアップロードしておくこと
"""
import pandas as pd

tweet_df = pd.read_csv("/content/ツイートデータ_整理_空白削除.csv")
tweet_df["ネガ"] = pd.to_numeric(tweet_df["ネガ"] , downcast='integer')
tweet_df["ポジ"] = pd.to_numeric(tweet_df["ポジ"] , downcast='integer')

#ネガ、ネガ以外で1000件ずつ取得する
#
nega_df = tweet_df[tweet_df["ネガ"] == 1.0]
nega_df = nega_df[0:1000]
nega_df

no_nega_df = tweet_df[tweet_df["ネガ"] == 0.0]
no_nega_df = no_nega_df[0:1000]
no_nega_df
tweet_df = pd.concat([nega_df, no_nega_df])
tweet_df = tweet_df.reset_index(drop=True)
tweet_df

#教師データ用に整形
train_text_batch = nega_df['ツイート'].values.tolist()
train_text_batch.extend(tweet_df['ツイート'].values.tolist())

train_labels = nega_df['ネガ'].values.tolist()
train_labels.extend(tweet_df['ネガ'].values.tolist())
"""

#本来は、ここで前処理を実施するべきだが、処理の流れを追っていく目的があるため省く

In [ ]:
import collections
#ここでラベルの割合を確認する
c = collections.Counter(train_labels)
print(c)

In [ ]:

#日本語の事前学習モデルを読み込む
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking', return_dict=True)

# 訓練モードに変更
model.train() 

#GPUモードにする
model.to(device)

In [ ]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5) # AdamWオプティマイザ


In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')


#トークンに分割する(tokenizer内部でmecab使った形態素の処理をしている)
#bertでは、文頭に[CLS]、分の終わりに[SEP]が必要だが、以下の処理では、文章を入れるだけで[CLS]、[SEP]を設定してくれる
#さらに、paddingをTrueにすることで、文字列が最も長い文に合わせて配列の大きさを調整する処理もやってくれるので便利だ
encoding = tokenizer(train_text_batch, return_tensors='pt', padding=True, truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

encoding

In [ ]:

"""
tokenized_text = tokenizer.tokenize(train_text_batch[0])
print(tokenized_text)

indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
print(indexed_tokens)

print(tokenizer.convert_tokens_to_ids(["[CLS]"]))
print(tokenizer.convert_tokens_to_ids(["[SEP]"]))
"""

In [ ]:
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
from sklearn.model_selection import train_test_split

#tensor型に変換
input_ids = torch.tensor(input_ids)
attention_mask = torch.tensor(attention_mask)
train_labels = torch.tensor(train_labels)

# データセットクラスの作成
# なんでTensorDataset使ってデータを分割しているかって？
# 分割しなくても良いけど、その場合、メモリが16G程度だとすぐにRAMがいっぱいになてしまうんだ
dataset = TensorDataset(input_ids, attention_mask, train_labels)


#トレーニングデータ、評価データを9:1の割合で分割する
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# データセットを訓練用と検証用に分ける
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
print(train_dataset[0])
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100,
#                                                    stratify=y)

print('訓練データ数：{}'.format(train_size))
print('検証データ数:　{} '.format(val_size))

# データローダーの作成
batch_size = 32

# 訓練データローダー
train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), # ランダムにデータを取得してバッチ化
            batch_size = batch_size
        )

# 検証データローダー
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset), # 順番にデータを取得してバッチ化
            batch_size = batch_size
        )

In [ ]:
input_ids

In [ ]:
# 訓練パートの定義
def train(model):
    model.train() # 訓練モードで実行
    train_loss = 0


    for batch in train_dataloader:# train_dataloaderはword_id, mask, labelを出力する点に注意
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        optimizer.zero_grad()
        outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        #outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        #log_str = "loss値は「%s」" % str(loss.item())
        #print(log_str)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_loss += loss.item()
    return train_loss

# テストパートの定義
def validation(model):
    model.eval()# 検証モードに切り替え
    val_loss = 0
    with torch.no_grad(): # 訓練じゃないので勾配を計算しない
        for batch in validation_dataloader:
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            with torch.no_grad():        
                (loss, logits) = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels)
            val_loss += loss.item()
    return val_loss

In [ ]:
# 学習の実行
max_epoch = 20
train_loss_ = []
test_loss_ = []

#エポック数の設定
for epoch in range(max_epoch):
    train_ = train(model)
    #test_ = train(model)
    train_loss_.append(train_)
    #test_loss_.append(test_)

    print(train_)

In [ ]:
model.eval()# 訓練モードをオフ
for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():   
        # 学習済みモデルによる予測結果をpredsで取得     
        preds = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        print(f'出力:{preds}')

In [ ]:
# データの比較しやすい様にpd.dataframeへ整形する
import pandas as pd
import numpy as np
# pd.dataframeへ変換（GPUに乗っているTensorはgpu->cpu->numpy->dataframeと変換）
logits_df = pd.DataFrame(preds[0].cpu().numpy(), columns=['logit_0', 'logit_1'])

## np.argmaxで大き方を予測の正解とする
pred_df = pd.DataFrame(np.argmax(preds[0].cpu().numpy(), axis=1), columns=['pred_label']) #予測したラベル
label_df = pd.DataFrame(b_labels.cpu().numpy(), columns=['true_label']) #正解のラベル

#！評価としては本来は、AUCなど使ったほうが良い
accuracy_df = pd.concat([logits_df, pred_df, label_df], axis=1)

accuracy_df.head()

In [ ]:
#作成したモデルを保存する

#参考サイト
#https://tzmi.hatenablog.com/entry/2020/03/05/222813

#GPUバージョンで保存
model_path = 'gpu_model.pth'
torch.save(model.state_dict(), model_path)

#念の為読み込めることを確認しておく
model.load_state_dict(torch.load(model_path))



#CPUバージョンでも保存(ローカルのPCでも使えるようにしておく)
model_path = 'cpu_model.pth'
torch.save(model.to('cpu').state_dict(), model_path)